In [1]:
### INITIALIZATION

### Importing standard modules and date-special modules:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
### Constants declaring:
All = slice(None) ### Internal constant for slicing
arr_tup_index = [(All, All, All), (All, All, ['DM', 'EM', 'FM']), (All, All, 'DM'), (All, All, 'EM'), (All, All, 'FM')] ### Regions' dataset filters
arr_tup_market = ['All', 'ISON', 'DM', 'EM', 'FM'] ### Regions' external names
num_rating_round_level = 2 ### Rating rounding order
num_quantile_bins = 10 ### Bins number for data distribution
arr_roll_max = [12] ### Length of look-back variants in months #[12, 36, 60]
path_source = 'Data_Files/Test_Files/Quick_Visualization_Test_Source.xlsx' ### Path for MS Excel source file
path_result = 'Data_Files/Test_Files/Quick_Visualization_Test_Table.xlsx' ### Path for MS Excel result file

In [2]:
### DEFINING CONSTANT RATING PERIODS LENGTH CALCULATOR

def rolling_count(iter_delta):
    if (iter_delta == 0):
        rolling_count.iter_counter = rolling_count.iter_counter + 1
        rolling_count.iter_result = 0
    else:
        rolling_count.iter_result = rolling_count.iter_counter
        rolling_count.iter_counter = 1
    return rolling_count.iter_result
rolling_count.iter_counter = 1 
rolling_count.iter_result = 0

In [ ]:
### MAIN SCRIPT

### Source data extraction:
df_source_raw = pd.read_excel(path_source, index_col = [0, 1, 2], parse_dates = True, 
                              na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                           '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)

### Resulting table initialazition:
arr_parameters = ['Average x-sectional median', 
                  'Average x-sectional range (90 – 10 ptile)', 
                  'Proportion of revised observations (decile bins)', 
                  'Average revisions per country', 
                  'Average age of ratings (months)', 
                  'Mean revision abs step (decile bins)', 
                  'Mean % of x-section revised in past 12 mo (decile bins)']
idx_table_res = pd.MultiIndex.from_product([arr_parameters, arr_tup_market], names = ['Parameter', 'Region'])
df_table_res = pd.DataFrame(index = idx_table_res, columns = df_source_raw.columns)

### Main loop performing:
for iter_pillar_num, iter_pillar in enumerate(df_source_raw.columns): ### Agencies iterating
    for iter_market_num, iter_tup_market in enumerate(arr_tup_index):  ### Universe filters iterating
        ### Grades series defining:
        iter_ser_grades = df_source_raw.loc[iter_tup_market, iter_pillar].dropna().sort_index(level = ['Country', 'Date'], ascending = [True, True])        
        ### Counter for country rankings:
        iter_ser_counter = iter_ser_grades.groupby('Country').count()
        ### Quantile bins distribution:
        iter_ser_deciles = df_source_raw.loc[All, iter_pillar].groupby('Date').transform(lambda iter_group: pd.qcut(iter_group, q = num_quantile_bins, 
                                                                                                                     labels = False, duplicates = 'drop')) 
        ### Bin changing fixation:
        iter_ser_deltas = iter_ser_deciles.dropna().groupby('Country').diff().sort_index(level = ['Country', 'Date'], ascending = [True, True])
        ### Rolling revisions number for every date calculation:
        arr_df_roll_flag_date = {} ### Cross-sectional changes quantity for each look-back period
        arr_df_roll_flag_market = {} ### Cros-sectional changes proportion for each look-back period
        arr_df_crosstab = {}        
        for iter_roll_max in arr_roll_max:
            num_roll_max = iter_roll_max
            num_roll_min = 2 # num_roll_max // 2
            ### Bin changing signed flag:
            iter_ser_flag = iter_ser_deltas.copy()
            iter_ser_flag[iter_ser_flag != 0] = iter_ser_flag / iter_ser_flag.abs()
            iter_ser_flag_plus = iter_ser_flag.copy()
            iter_ser_flag_plus[iter_ser_flag_plus != 1] = 0
            iter_ser_flag_minus = iter_ser_flag.copy()
            iter_ser_flag_minus[iter_ser_flag_minus != -1] = 0
            ### Positive changes quantity for country for look-back period:
            iter_ser_roll_flag_plus = iter_ser_flag_plus.groupby('Country').rolling(window = num_roll_max, min_periods = num_roll_min).sum()
            iter_ser_roll_flag_plus = iter_ser_roll_flag_plus.reset_index(level = 0, drop = True)
            ### Negative changes quantity for country for look-back period:            
            iter_ser_roll_flag_minus = iter_ser_flag_minus.abs().groupby('Country').rolling(window = num_roll_max, min_periods = num_roll_min).sum()
            iter_ser_roll_flag_minus = iter_ser_roll_flag_minus.reset_index(level = 0, drop = True)
            ### Any changes quantity for country for look-back period:
            iter_ser_roll_flag = iter_ser_roll_flag_plus + iter_ser_roll_flag_minus
            ### Changes quantity info concatenation:
            iter_df_roll_flag = pd.concat([iter_ser_roll_flag, iter_ser_roll_flag_plus, iter_ser_roll_flag_minus], axis = 1)
            iter_df_roll_flag.sort_index(level = ['Country', 'Date'], ascending = [True, True], inplace = True)  
            iter_df_roll_flag = iter_df_roll_flag.loc[iter_tup_market, All]
            iter_df_roll_flag.columns = ['Any', 'Plus', 'Minus']
            ### Changes quantity for cross-section for look-back period:
            iter_df_roll_flag_date = iter_df_roll_flag.groupby('Date').sum()          
            arr_df_roll_flag_date[num_roll_max] = iter_df_roll_flag_date
            ### Changes proportion for cross-section for look-back period:
            iter_df_roll_flag_market_all = iter_df_roll_flag.groupby('Date')
            iter_df_roll_flag_market_revised = iter_df_roll_flag[iter_df_roll_flag['Any'] > 0].groupby('Date')
            iter_df_roll_flag_market = iter_df_roll_flag_market_revised.count().div(iter_df_roll_flag_market_all.count())['Any']
            iter_df_roll_flag_market.fillna(0, inplace = True)
            arr_df_roll_flag_market[num_roll_max] = iter_df_roll_flag_market            
        ### Constant periods lengths series calculation:
        iter_ser_deltas = iter_ser_deltas.shift(-1)    
        iter_ser_distances = iter_ser_deltas.apply(rolling_count)
        iter_ser_distances = iter_ser_distances.loc[iter_tup_market]
        iter_ser_deltas = iter_ser_deltas.shift()  
        iter_ser_deltas = iter_ser_deltas.loc[iter_tup_market]
        ### Deltas number for countries calculation:        
        iter_ser_deltas_countries = iter_ser_deltas.dropna()[iter_ser_deltas != 0].groupby('Country').count()
        iter_ser_deltas_countries = iter_ser_deltas_countries / iter_ser_deltas.dropna().groupby('Country').count()
        iter_ser_deltas_countries.fillna(0, inplace = True)
        iter_ser_deltas_countries = iter_ser_deltas_countries * iter_ser_deltas.dropna().groupby('Country').count()       
        iter_ser_deltas_countries.name = 'Country revisions number'
        ### Mean deltas for universe filter:
        iter_ser_deltas_mean = pd.Series(iter_ser_deltas_countries.mean(), index = iter_ser_deltas_countries.index)        
        iter_ser_deltas_mean.name = iter_pillar + ' / ' + arr_tup_market[iter_market_num] + ' mean'
        ### Timeline for filtered market time-series and cross-sections aggregating:
        grouper_market_by_country = iter_ser_grades.groupby('Country')
        grouper_market_by_date = iter_ser_grades.groupby('Date')        
        ### Plots drawing:
        if ((iter_pillar_num == 0) & (iter_market_num == 0)):
            ### Filtered universe grades distribution drawing:            
            (iter_ser_grades.groupby(pd.cut(iter_ser_grades, 20)).count() / iter_ser_grades.count()).plot(figsize = (15, 5), kind = 'bar', color = '#1f77b4',
                                                                                    title = iter_pillar + ' / ' + arr_tup_market[iter_market_num] + ' - distribution')
            plt.show()        
            ### Timeline for filtered countries plotting:
            dict_color = {'boxes': 'DarkBlue', 'medians': 'Green', 'whiskers': 'Blue', 'caps': 'Black'}
            iter_ser_grades.unstack('Country').reindex(iter_ser_grades.unstack('Country').median().sort_values().index, axis = 1).plot.box(figsize = (20, 5), 
                                                                               title = iter_pillar + ' / ' + arr_tup_market[iter_market_num] + ' - by country box plot',
                                                                               color = dict_color, sym = 'r+', whis = [10, 90])        
            plt.show()
            ### Revisions number for filtered countries plotting:        
            iter_ser_deltas_countries.plot.bar(figsize = (20, 5), title = iter_pillar + ' / ' + arr_tup_market[iter_market_num] + ' - by country revisions number',
                                               legend = True, width = 0.5)
            iter_ser_deltas_mean.plot(color = 'Orange', legend = True)
            plt.show()
        ### Resulting table filling:
        df_table_res.loc[('Proportion of revised observations (decile bins)', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                '{:.0%}'.format(len(iter_ser_deltas.dropna()[iter_ser_deltas != 0].index) / iter_ser_grades.count())        
        df_table_res.loc[('Average revisions per country', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                round(iter_ser_deltas_countries.mean(), 0)
        df_table_res.loc[('Average age of ratings (months)', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                round(iter_ser_distances[iter_ser_distances > 0].mean(), 1)
        df_table_res.loc[('Mean revision abs step (decile bins)', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                round(iter_ser_deltas.abs()[iter_ser_deltas != 0].mean(), 1)       
        df_table_res.loc[('Average x-sectional range (90 – 10 ptile)', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                round((grouper_market_by_date.quantile(0.90) - grouper_market_by_date.quantile(0.10)).mean(), 1)
        df_table_res.loc[('Average x-sectional median', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                round(grouper_market_by_date.median().mean(), 1)        
        for iter_roll_max in arr_roll_max:
            if (iter_roll_max == 12):
                num_roll_max = iter_roll_max
                iter_df_roll_flag_date = arr_df_roll_flag_date[num_roll_max]
                iter_df_roll_flag_market = arr_df_roll_flag_market[num_roll_max]
                df_table_res.loc[('Mean % of x-section revised in past 12 mo (decile bins)', arr_tup_market[iter_market_num]), iter_pillar] = \
                                                        '{:.0%}'.format(round(iter_df_roll_flag_market.mean(), 2))
#        break        
#    break
#df_table_res.to_excel(path_result, engine = 'openpyxl')
df_table_res.to_excel(path_result) 

In [16]:
### TEMP

